# Data Cleaning

First of all we have to import the basic python libaries.

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'



## Read the raw data files
The project provides 3 three files as described in the dataset section. The files are formated json, we use the pandas read_json method to create dataframes.

In [2]:
# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## Portfolio

We start to clean the portfolio dataset. Let's display the first columns to see how the data looks in a dataframe.

In [3]:
portfolio.columns.name = 'index'
portfolio.index.name = None
portfolio.head(3)


index,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed


### Portfolio Assessing Data
* One Hot encoding channels: The channels used to reeive the offers are organized in list datatypes. This should be changed. I will do split the channels, that I get an own column for each channel.
* Add an alternative column with an offer_name build from type_reward_difficulty_duration
* Change duration values from days to hours. In later coming preprocessing steps I have to combine the timestamp of events with the offer duration. Timestamps are in hours so I will change the duration also to hours.
* Change column name id to offer_id. This is also for harmonization with the other datasets, to simplify an later merging process.
* Add one hot encoded vector for offer_type columns to reduce later preprocessing steps.

Before I start I create a copy from portfolio dataframe where I can do my changes.


In [4]:
# Create a copy to keep the original dataset
clean_portfolio = portfolio.copy()

#### One Hot Encoding for channels column

In [5]:
# One hot encoding channels
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

clean_portfolio = clean_portfolio.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(clean_portfolio.pop('channels')),
                index=portfolio.index,
                columns=mlb.classes_))

clean_portfolio.loc[:, ['email', 'mobile', 'social', 'web']] = clean_portfolio.loc[:, ['email', 'mobile', 'social', 'web']].astype(int) 

In [6]:
clean_portfolio.info()
clean_portfolio.columns.name='index'
clean_portfolio.iloc[:3,5:]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype           
---  ------      --------------  -----           
 0   reward      10 non-null     int64           
 1   difficulty  10 non-null     int64           
 2   duration    10 non-null     int64           
 3   offer_type  10 non-null     object          
 4   id          10 non-null     object          
 5   email       10 non-null     Sparse[int64, 0]
 6   mobile      10 non-null     Sparse[int64, 0]
 7   social      10 non-null     Sparse[int64, 0]
 8   web         10 non-null     Sparse[int64, 0]
dtypes: Sparse[int64, 0](4), int64(3), object(2)
memory usage: 924.0+ bytes


index,email,mobile,social,web
0,1,1,1,0
1,1,1,1,1
2,1,1,0,1


In [7]:
clean_portfolio.columns.name = 'index'
clean_portfolio.index.name = None
clean_portfolio.iloc[:3]

index,reward,difficulty,duration,offer_type,id,email,mobile,social,web
0,10,10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0
1,10,10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1
2,0,0,4,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1


Now the channels column is replace and I get for each channel an seperate column with a null one declaration.

#### Create column called with alternative offer name
The column will be named as 'ticks'

In [8]:
# There will come several plots where the offer_id is on an axis a label
# The offer_id tells us nothing about the offer itself. 
# I create a column ticks with concatenate offer specific feature names
clean_portfolio['ticks'] = clean_portfolio.offer_type+'_'+clean_portfolio.reward.astype(str)\
                        +'_'+clean_portfolio.difficulty.astype(str)\
                        +'_'+(clean_portfolio.duration).astype(int).astype(str)




In [9]:

clean_portfolio.iloc[:3,-5:]

index,email,mobile,social,web,ticks
0,1,1,1,0,bogo_10_10_7
1,1,1,1,1,bogo_10_10_5
2,1,1,0,1,informational_0_0_4


The alternative column name is added as additinal column 'ticks'

#### Change format of duration from days to hours

In [10]:
# Change duration values from days to hours
clean_portfolio.duration = clean_portfolio.duration * 24

In [11]:
clean_portfolio.head(3)

index,reward,difficulty,duration,offer_type,id,email,mobile,social,web,ticks
0,10,10,168,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0,bogo_10_10_7
1,10,10,120,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,bogo_10_10_5
2,0,0,96,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1,informational_0_0_4


#### Rename columns id to offer_id

In [12]:
# Change column name id to offer_id
clean_portfolio = clean_portfolio.rename(columns={'id': 'offer_id'})
clean_portfolio.head(3)

index,reward,difficulty,duration,offer_type,offer_id,email,mobile,social,web,ticks
0,10,10,168,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0,bogo_10_10_7
1,10,10,120,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,bogo_10_10_5
2,0,0,96,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1,informational_0_0_4


#### One hot encode vector for offer_type column

In [13]:
clean_portfolio = clean_portfolio.join(pd.get_dummies(clean_portfolio.offer_type))
clean_portfolio.head()

,reward,difficulty,duration,offer_type,offer_id,email,mobile,social,web,ticks,bogo,discount,informational
0,10,10,168,bogo,ae264e3637204a6fb9bb56bc8210ddfd,1,1,1,0,bogo_10_10_7,1,0,0
1,10,10,120,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1,1,1,1,bogo_10_10_5,1,0,0
2,0,0,96,informational,3f207df678b143eea3cee63160fa8bed,1,1,0,1,informational_0_0_4,0,0,1
3,5,5,168,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1,1,0,1,bogo_5_5_7,1,0,0
4,5,20,240,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1,0,0,1,discount_5_20_10,0,1,0


#### Write the cleaned portfolio to a file
I write the cleaned portfolio data to a csv file, that I can use it in further notebooks

In [14]:
clean_portfolio.to_csv('data/clean_portfolio.csv', index=False)

## Profile
Let's display the first columns to see how the data looks in a dataframe.


In [15]:
profile.columns.name='index'
profile.head(3)

index,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN


### Profile Assessing data

* convert became_member_on to datatime format
* Add columns for year, month, day, member_since_days in relation to today
* Modify features from users with no gender no income and age 118 years to gender 'U' for unknown and income and age to mean values from the other customers.
* Change column name id to person_id
* Add One hot Encoding columns for gender columns, to reduce later preprocessing steps

In [16]:
# create a copy to keepthe original dataframe
clean_profile = profile.copy()

profile.head(3)

index,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN


#### Convert became_member_on column

In [17]:
# convert became_member_on column to datetime
clean_profile.became_member_on = pd.to_datetime(clean_profile.became_member_on, format='%Y%m%d')
clean_profile.head(3)


index,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN


#### Add columns from become_member_on column

In [18]:
# Add columns for year, month, day, member_since_days in relation to today
clean_profile.loc[:, 'year'] = clean_profile.became_member_on.dt.year
clean_profile.loc[:, 'month'] = clean_profile.became_member_on.dt.month
clean_profile.loc[:, 'day'] = clean_profile.became_member_on.dt.weekday
clean_profile.loc[:, 'member_since_days'] = (pd.to_datetime('today') - clean_profile.became_member_on).astype('timedelta64[D]').astype(int)

clean_profile.head(3)

index,gender,age,id,became_member_on,income,year,month,day,member_since_days
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,2017,2,6,1700
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,2017,7,5,1547
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,2018,7,3,1185


In [19]:
clean_profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   gender             14825 non-null  object        
 1   age                17000 non-null  int64         
 2   id                 17000 non-null  object        
 3   became_member_on   17000 non-null  datetime64[ns]
 4   income             14825 non-null  float64       
 5   year               17000 non-null  int64         
 6   month              17000 non-null  int64         
 7   day                17000 non-null  int64         
 8   member_since_days  17000 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(2)
memory usage: 1.2+ MB


In [20]:
clean_profile.iloc[:3,[3, 5,6,7,8]]

index,became_member_on,year,month,day,member_since_days
0,2017-02-12,2017,2,6,1700
1,2017-07-15,2017,7,5,1547
2,2018-07-12,2018,7,3,1185


#### Replace gender nan values

To have a possibility to use the customer group who does not give demographic features I fill this with dummy values. For gender I define 'U' for unknown.

In [21]:
# Modify gender column NaN values to 'U' for unknown
clean_profile.gender = clean_profile.gender.fillna('U')
clean_profile.head(3)

index,gender,age,id,became_member_on,income,year,month,day,member_since_days
0,U,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,2017,2,6,1700
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,2017,7,5,1547
2,U,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,2018,7,3,1185


#### Fill missing income values

The missing income values I fill with the mean income.

In [22]:
# Modify income columns NaN values with mean Income 
clean_profile.income = clean_profile.income.fillna(clean_profile.income.mean())
clean_profile.head(3)

index,gender,age,id,became_member_on,income,year,month,day,member_since_days
0,U,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,65404.991568,2017,2,6,1700
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.000000,2017,7,5,1547
2,U,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,65404.991568,2018,7,3,1185


#### Modify the age of customers with age = 118 to the mean age

In [23]:
# Modify age values 118 to mean values
# Get the index of age == 118 customers
idx = clean_profile[clean_profile.age == 118].index
# calculate mean for customers age != 118
mean = clean_profile.loc[~clean_profile.index.isin(idx)].age.mean()
# Replace age 118 by mean age
clean_profile.loc[idx, 'age'] = mean
clean_profile.head(3)

index,gender,age,id,became_member_on,income,year,month,day,member_since_days
0,U,54.393524,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,65404.991568,2017,2,6,1700
1,F,55.000000,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.000000,2017,7,5,1547
2,U,54.393524,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,65404.991568,2018,7,3,1185


#### Change column name id to person_id

In [24]:
clean_profile = clean_profile.rename(columns={'id': 'person_id'})
clean_profile.head(3)

index,gender,age,person_id,became_member_on,income,year,month,day,member_since_days
0,U,54.393524,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,65404.991568,2017,2,6,1700
1,F,55.000000,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.000000,2017,7,5,1547
2,U,54.393524,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,65404.991568,2018,7,3,1185


#### Add columns for each gender as one hot encoding

In [25]:
clean_profile = clean_profile.join(pd.get_dummies(clean_profile.gender))

clean_profile.iloc[:3,-4:]

,F,M,O,U
0,0,0,0,1
1,1,0,0,0
2,0,0,0,1


In [26]:
clean_profile.columns.name='index'
clean_profile.iloc[:3,-4:]

index,F,M,O,U
0,0,0,0,1
1,1,0,0,0
2,0,0,0,1


#### Write cleaned profile data to csv file

In [27]:
clean_profile.to_csv('data/clean_profile.csv', index=False)

### Transcript

Let's have a look on the dataframe


In [28]:
transcript.columns.name='index'
transcript.head(3)

index,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0


### Assessing Transcript data

* Rename column name person to person_id
* One hot encoding of column event
* Clear column value
* Drop duplicate rows after normalizing value column

In [29]:
# Create a copy to keep the original dataframe
clean_transcript = transcript.copy()

#### Rename person column

In [30]:
# Rename column person to person_id
clean_transcript = clean_transcript.rename(columns={'person': 'person_id'})

#### One hot encoding for event column

In [31]:
# One Hot Encoding column event
clean_transcript = pd.get_dummies(data=clean_transcript, columns=['event'])

In [32]:
clean_transcript.head(3)

,person_id,value,time,event_offer completed,event_offer received,event_offer viewed,event_transaction
0,78afa995795e4d85b5d9ceeca43f5fef,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0,0,1,0,0
1,a03223e636434f42ac4c3df47e8bac43,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0,0,1,0,0
2,e2127556f4f64592b11af22de27a7932,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0,0,1,0,0


#### Normalize the value column

The value columns contains json data. We can use the pandas json_normalize method. After we do this, we get two columns `offer_id` and `offer id`. These two columns with same variable but different column names must be merged in one column.
The additional column reward can be droped, because this column is part of the portfolio dataset.


In [33]:
# Normalize column value
normalized = pd.json_normalize(data=clean_transcript.value)
# Index where column 'offer id' has values
idx_1 = normalized['offer id'].dropna().index
# Set values from column 'offer id' in column 'offer_id'
normalized.loc[idx_1, 'offer_id'] = normalized.loc[idx_1, 'offer id']
# Drop columns 'offer id' and 'reward'
normalized = normalized.drop(columns=['offer id', 'reward'])
# Join normalize dataframe with transcript
clean_transcript = clean_transcript.join(normalized)
# Drop column value from clean_trancript, no longer needed
clean_transcript = clean_transcript.drop(columns='value')

In [34]:
clean_transcript.head(3)

,person_id,time,event_offer completed,event_offer received,event_offer viewed,event_transaction,amount,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,0,0,1,0,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,NaN,2906b810c7d4411798c6938adc9daaa5


#### Drop duplicate rows

In [35]:
# Drop duplicate rows
clean_transcript = clean_transcript.drop_duplicates()

#### Rename columns from one hot hot encoding offer_type

In [36]:
clean_transcript = clean_transcript.rename(columns={'event_offer received': 'received', 
                         'event_offer viewed':'viewed', 
                         'event_offer completed':'completed',
                         'event_transaction': 'transaction'})



In [37]:
clean_transcript.columns.name='index'
clean_transcript.query('transaction == 1')

clean_transcript.iloc[[1,2,12657], :]

index,person_id,time,completed,received,viewed,transaction,amount,offer_id
1,a03223e636434f42ac4c3df47e8bac43,0,0,1,0,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,0,0,1,0,0,NaN,2906b810c7d4411798c6938adc9daaa5
12657,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,0,0,0,1,34.56,NaN


In [38]:
transcript.event.value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

#### Write the cleaned datafraame to csv file. 

In [39]:
clean_transcript.to_csv('data/clean_transcript.csv', index=False)

### Merge all dataframes

In [40]:
clean_portfolio.columns

Index(['reward', 'difficulty', 'duration', 'offer_type', 'offer_id', 'email',
       'mobile', 'social', 'web', 'ticks', 'bogo', 'discount',
       'informational'],
      dtype='object')

In [41]:
# Full Dataframe = Merge Profile and clean_transcript
full_df = clean_profile.merge(clean_transcript, how='outer', left_on='person_id', right_on='person_id')
# Merge Full Dataframe with Portfolio 
full_df = full_df.merge(clean_portfolio, how='outer', left_on='offer_id', right_on='offer_id')

### Assessing full_df
* Add columns validity from time + duration

In [42]:
full_df.duration

0         168.0
1         168.0
2         168.0
3         168.0
4         168.0
          ...  
306132    120.0
306133    120.0
306134    120.0
306135    120.0
306136    120.0
Name: duration, Length: 306137, dtype: float64

In [43]:
full_df['validity'] = full_df.time + full_df.duration

In [44]:
full_df.columns.name = 'index'
full_df.loc[:3, ['time', 'duration', 'validity']]

index,time,duration,validity
0,168,168.0,336.0
1,216,168.0,384.0
2,0,168.0,168.0
3,18,168.0,186.0


#### Write full dataframe to csv file.

In [45]:
# Save full_df to csv
full_df.to_csv('data/full.csv', index=False)